In [1]:
# Import package, test suite, and other packages as needed
import sys, pytest, os, py3Dmol, json
import occupancy_fingerprinter
from occupancy_fingerprinter import BindingSite
from occupancy_fingerprinter import Grid
import numpy as np
import mdtraj as md
import h5py as h5
from copy import deepcopy
from datetime import datetime
from pathlib import Path
from utils import *
cwd = Path.cwd()
mod_path = Path(os.getcwd()).parent

# Build param dict 
param_dict = {}

Make the Super Trajectory
=========================
    * traj_dir (str): 
        Specify the working directory where subdirectories name 'pdb' and 'dcd' can be found with the appropriate topologies/trajectories to include for occupany fingerprinting
        
    * resSeqs_to_include (np.array): 
        Array of resSeqs to include in the super_trajectory

    *binding_pocket_resSeqs (np.array):
            Array of binding pocket residues to use for alignment
        
    * super_traj_dir (str): 
        String path to output directory where super_traj will be stored

In [2]:
# Specify traj directory
traj_dir = '/expanse/lustre/projects/iit122/dcooper/CB2/PTwFR/resampled/' # CHANGE AS NEEDED
resSeqs_to_include = np.arange(25,311) # CHANGE AS NEEDED
binding_pocket_resSeqs = np.array([25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293])
super_traj_dir = '/expanse/lustre/projects/iit122/dcooper/CB2/PTwFR/super_traj/all_atoms' # CHANGE AS NEEDED

"""
Do not change below this line
"""

# Update parameters
param_dict["traj_dir"] = traj_dir
param_dict["resSeqs_to_include"] = list(resSeqs_to_include)
param_dict["binding_pocket_resSeqs"] = list(binding_pocket_resSeqs)
param_dict["super_traj_dir"] = super_traj_dir

# Combine trajectories
traj, frame_labels = combine_trajectories(traj_dir, resSeqs_to_include, binding_pocket_resSeqs, super_traj_dir)


centroid_1.pdb (4531,)
centroid_10.pdb (4531,)
centroid_11.pdb (4531,)
centroid_12.pdb (4531,)
centroid_13.pdb (4531,)
centroid_14.pdb (4531,)
centroid_15.pdb (4531,)
centroid_16.pdb (4531,)
centroid_17.pdb (4531,)
centroid_18.pdb (4531,)
centroid_19.pdb (4531,)
centroid_2.pdb (4531,)
centroid_20.pdb (4531,)
centroid_3.pdb (4531,)
centroid_4.pdb (4531,)
centroid_5.pdb (4531,)
centroid_6.pdb (4531,)
centroid_7.pdb (4531,)
centroid_8.pdb (4531,)
centroid_9.pdb (4531,)


Occupancy Fingerprinting
=========================

Step 1. Preparing the Grid
------------------

    * bp_resSeqs (np.array): 
        Set some binding pocket residues to get the center of mass between. 
        
    * radius (int):
        1/2 width of the grid (Angstrom).
        
    * spacing (np.array):
        Array of the resolution of the grid in the x,y,z dimensions. 


In [3]:
# Identify binding site from trajectory
bp_resSeqs = np.array([87, 183, 91, 117, 94, 184, 113, 194, 265, 258]) # CHANGE AS NEEDED
radius = 10 # CHANGE AS NEEDED
spacing = np.array([0.5, 0.5, 0.5]) # CHANGE AS NEEDED

"""
Do not change below this line
"""
# Update parameters
param_dict["bp_resSeqs"] = list(bp_resSeqs)
param_dict["radius"] = int(radius)
param_dict["spacing"] = list(spacing)

# Get center
sele = traj.topology.select('name CA and resSeq '+ ' '.join([str(resSeq) for resSeq in bp_resSeqs]))
com = md.compute_center_of_mass(traj.atom_slice(sele)) * 10 # convert to Angstrom

# Build Grid object with binding site
g = Grid(traj)
center = np.array([com[:,0].mean(), com[:,1].mean(), com[:,2].mean()])
g.add_binding_site(center=center, 
                   r=radius, 
                   spacing=spacing)

# View protein
view = py3Dmol.view()
view.setBackgroundColor('white')
view.addModel(open(os.path.join(super_traj_dir, 'super_traj.pdb'), 'r').read(), 'pdb')
view.setStyle({'model':0}, {'cartoon': {'color':'grey'}})

# View grid
for x in g._sites[0]._grid_x[::4]:
    for y in g._sites[0]._grid_y[::4]:
        for z in g._sites[0]._grid_z[::4]:
            view.addSphere({'center': {'x': x, 'y': y, 'z': z}, 'radius': 0.3, 'color': 'red', 'opacity': 0.5})
view.zoomTo()
view.show()


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Step 2. Setting up Fingerprinting (won't be executed here)
------------------

    * fp_output_dir (str):
        String path to directory to store fingerprinting output

    * n_tasks (int):
        Number of cores to use for fingerprinting. 
        

In [4]:
fp_output_dir = '/expanse/lustre/projects/iit122/dcooper/CB2/PTwFR/super_traj/all_atoms/' # CHANGE AS NEEDED
n_tasks = 128 # CHANGE AS NEEDED

""" 
Do not change below this line
"""
# Update parameters
param_dict["fp_output_dir"] = fp_output_dir
param_dict["n_tasks"] = n_tasks

Stratification Sampling
=========================


Clustering using stratification sampling (see Xie et al., JCIM, 2018) 

Parameters
----------
    * docking_dir (str):
        String path to docking dir where subdirectproes will be built

    * ligands (dict):
        Dictionary where the keys are ligand names and the items are smiles strings.

    * n_clusters (np.array):
        Array containing different n_clusters to try with hierarchical clustering.

    * save_dir (str):
        
    

In [5]:
# Set directors
docking_dir = '/expanse/lustre/projects/iit122/dcooper/CB2/docking/' # CHANGE AS NEEDED
ligands = {'CP55490':  'CCCCCCC(C)(C)C1=CC(=C(C=C1)[C@@H]2C[C@@H](CC[C@H]2CCCO)O)O',
           'HU308': 'CCCCCCC(C)(C)C1=CC(=C(C(=C1)OC)[C@@H]2C=C([C@@H]3C[C@H]2C3(C)C)CO)OC',
           'GP1a': 'CC1=CC2=C(C=C1)C3=C(C2)C(=NN3C4=C(C=C(C=C4)Cl)Cl)C(=O)NN5CCCCC5',
           '9THC': 'CCCCCC1=CC(=C2[C@@H]3C=C(CC[C@H]3C(OC2=C1)(C)C)C)O',
           'GW405833': 'CC1=C(C2=C(N1C(=O)C3=C(C(=CC=C3)Cl)Cl)C=CC(=C2)OC)CCN4CCOCC4',
           'SR1445228': 'CC1=CC=C(C=C1)CN2C(=CC(=N2)C(=O)N[C@H]3[C@]4(CC[C@H](C4)C3(C)C)C)C5=CC(=C(C=C5)Cl)C',
           'AM1241': 'Ic2ccc(N(=O)=O)cc2C(=O)c(c4ccccc14)cn1CC3CCCCN3C',
           'AM1710': 'CCCCCCC(C)(C)C1=CC(=C2C(=C1)OC(=O)C3=C2C=C(C=C3)OC)O',
           'AM2232': 'C1=CC=C2C(=C1)C=CC=C2C(=O)C3=CN(C4=CC=CC=C43)CCCCC#N',
           'AM2233': 'CN1CCCCC1CN2C=C(C3=CC=CC=C32)C(=O)C4=CC=CC=C4I',
           'AM630': 'CC1=C(C2=C(N1CCN3CCOCC3)C=C(C=C2)I)C(=O)C4=CC=C(C=C4)OC',
           'JWH015': 'CCCN1C(=C(C2=CC=CC=C21)C(=O)C3=CC=CC4=CC=CC=C43)C',
           'JWH133': 'CCCC(C)(C)C1=CC2=C(C=C1)[C@@H]3CC(=CC[C@H]3C(O2)(C)C)C',
           'L759633': 'CCCCCCC(C)(C)C1=CC2=C([C@@H]3CC(=CC[C@H]3C(O2)(C)C)C)C(=C1)OC',
           'L759656': 'CCCCCCC(C)(C)C1=CC2=C([C@@H]3CC(=C)CC[C@H]3C(O2)(C)C)C(=C1)OC',
           'STS135': 'FCCCCCn1cc(c2c1cccc2)C(=O)NC12CC3CC(C2)CC(C1)C3',
           'UR144': 'CCCCCN1C=C(C2=CC=CC=C21)C(=O)C3C(C3(C)C)(C)C'} # CHANGE AS NEEDED
n_clusters = [i**2 for in in range(1, 10)] # CHANGE AS NEEDED
save_dir = os.path.join(docking_dir, 'stratification_results') # CHANGE AS NEEDED

""" 
Do not change below this line
"""

# Update parameters
param_dict["docking_dir"] = docking_dir
param_dict["ligands"] = ligands
param_dict["n_clusters"] = list(n_clusters)
param_dict["save_dir"] = save_dir

In [6]:
# Save dictionary to .json file for running
from utils import NumpyEncoder

json_fn = './run.json' # CHANGE AS NEEDED
with open(json_fn, 'w') as f:
    json.dump(param_dict, f, indent=4, cls=NumpyEncoder)
    f.close()

In [7]:
print('JSON WRITTEN')

JSON WRITTEN
